In [1]:
!pip install requests
!pip install qmpy-rester

In [2]:
import requests
import json
import qmpy_rester as qr
from concurrent.futures import ThreadPoolExecutor
import math

In [3]:
BASE_URL = "http://oqmd.org/oqmdapi/formationenergy?fields=name,entry_id,spacegroup,ntypes,band_gap,delta_e&limit=200&filter=element_set=O AND ntypes=3"
materials_data = []

In [4]:

while True:
    
    print(f"Getting data from URL: {_next}")
    
    r = requests.get(BASE_URL)
    _next = r.json().get('links').get('next')
    if _next is None:
        break
    
    data_list = r.json().get('data')
    materials_data += data_list
    BASE_URL = _next
    
# r.json()
# data = r.json().get('data')
print(len(materials_data))
# data


NameError: name '_next' is not defined

In [22]:
len(materials_data)

4200

In [5]:
## Return list of data
with qr.QMPYRester() as q:
    kwargs = {
        'element_set': 'O',      
        'ntypes': '3',
        'limit': 10,
        'fields': 'name,delta_e,band_gap'
        }

list_of_data = q.get_oqmd_phases(**kwargs)

Your filters are:
    limit=10000
    fields=name,delta_e,band_gap
    filter=element_set=O AND ntypes=3
Proceed? [Y/n]:y


In [6]:
str(list_of_data)[:2000]

'None'

In [7]:
def get_data(url):
    
    global materials_data
    
    print(f"Getting data from URL: {url}")
    
    r = requests.get(url)
    data_list = r.json().get('data')
    materials_data += data_list
    
    print(f"Done getting URL: {url}")

num_mate = 51051
limit_p_page = 500
offsets = math.ceil(num_mate / limit_p_page)
base_url = f"http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit={limit_p_page}&offset=[%OFFSET%]"

links = []
## Create links to visit
for i in range(offsets):
    url = base_url.replace('[%OFFSET%]', str(i*limit_p_page))
    links.append(url)

## Create a threadpool to get the data
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(get_data, links[:10])

print(len(materials_data))

Getting data from URL: http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit=500&offset=0
Getting data from URL: http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit=500&offset=500
Getting data from URL: http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit=500&offset=1000
Getting data from URL: http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit=500&offset=1500
Getting data from URL: http://oqmd.org/oqmdapi/formationenergy?fields=materials_data%name%2Centry_id%2Cspacegroup%2Cntypes%2Cband_gap%2Cdelta_e&filter=element_set%3DO+AND+ntypes%3D3&limit=

In [8]:
materials_data

[]

In [9]:
[i for i in materials_data if i['delta_e'] is None]

[]